In [1]:
import pickle
from sklearn import preprocessing
import numpy as np

In [2]:
# Load data
with open('msd_data1.pickle', 'rb') as fh1:
    msd_data = pickle.load(fh1)

doscaling = 1

if (doscaling == 1):
    xscaler = preprocessing.StandardScaler().fit(msd_data['X_train'])
    #standardize feature values
    X_train = xscaler.transform(msd_data['X_train'])
    X_test = xscaler.transform(msd_data['X_test'])
else:
    X_train = msd_data['X_train']
    X_test = msd_data['X_test']

Y_train = msd_data['Y_train']
Y_test = msd_data['Y_test']

In [3]:
msd_data

{'X_train': array([[  53.39967,   56.67781,   55.63508, ...,    1.3159 ,    2.37448,
            1.85999],
        [  42.83464,  -33.90478,   38.97704, ...,    9.72827, -175.51446,
          -15.8389 ],
        [  49.88591,   23.30563,   54.79012, ...,    1.94152,  101.27241,
            8.99355],
        ...,
        [  41.04397,  -96.30727,   56.8073 , ...,    3.60251,  -97.24809,
           -5.80797],
        [  42.8676 ,   43.36365,   29.17957, ...,   18.71014,  208.9029 ,
          -11.08426],
        [  36.57558,   29.8733 ,   -3.08101, ...,   21.0417 ,  106.60317,
            3.44396]]),
 'Y_train': array([1996., 2001., 2003., ..., 2009., 1982., 1981.]),
 'X_test': array([[  38.51784,   55.38723,   10.31652, ...,    6.66624,  -73.75585,
            9.47857],
        [  44.7309 ,   46.25776,    8.55636, ...,    4.5564 ,   53.49956,
            5.39116],
        [  43.08912,   45.70197,   17.18467, ...,  -13.64909,   -7.42137,
           -4.97266],
        ...,
        [  39.81814